# 1. Shortest Path in a Grid with Obstacles Elimination

You are given an m x n integer matrix grid where each cell is either 0 (empty) or 1 (obstacle). You can move up, down, left, or right from and to an empty cell in one step.

Return the minimum number of steps to walk from the upper left corner (0, 0) to the lower right corner (m - 1, n - 1) given that you can eliminate at most k obstacles. If it is not possible to find such walk return -1.

![img](https://assets.leetcode.com/uploads/2021/09/30/short1-grid.jpg)

Example 1:
```
Input: grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]], k = 1
Output: 6
Explanation: 
The shortest path without eliminating any obstacle is 10.
The shortest path with one obstacle elimination at position (3,2) is 6. Such path is (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (3,2) -> (4,2).
```

Overview

Like many grid search problems where the goal is to find the shortest path, the key to solve this problem is to apply the Breadth-First Search algorithm, as opposed to the Depth-First Search (DFS) algorithm. In this article, we will start with a classic BFS solution. Then on top of BFS, we will introduce a heuristic (greedy) strategy to speed up the algorithm, which eventually transforms our classic BFS algorithm into another classic algorithm called the A* search algorithm.

Approach 1: BFS (Breadth-First Search)

Intuition

In this problem, we must traverse the grid to reach a target cell, while the grid contains some obstacles. If the problem ends here, one could probably tell that this is a classic grid search problem, e.g. the problem of 1730. Shortest Path to Get Food fits the bill exactly.

However, the particularity of this problem is that one can eliminate obstacles to a certain extent. This constraint complicates our problem. First of all, if there were no limit on how many obstacles we can eliminate, then the shortest distance to reach the target cell would be the Manhattan distance between the starting cell and the target cell. Likewise, if the quota to eliminate the obstacles is greater than the Manhattan distance, then the shortest distance is guaranteed to be the Manhattan distance. However, we do have a limit on the number of obstacles that we can eliminate along the way. As a result, rather than taking a straightforward path to reach the target, we have to take some detour in certain cases, which implies that we need to explore all possible directions while respecting the constraint.

By exploring, we refer to the BFS strategy, rather than DFS. The BFS algorithm works like detecting an object with sonar. A sound wave propagages in all directions with equal speed. At any given moment, all the objects that the sound wave reaches have the same distance to the source of the sound. On the other hand, as soon as the sound wave reaches the object, the path is guaranteed to be the shortest, since the distance is proportional to the time, the more time it takes, the longer the distance is.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs.png)

Given the above intuition of the BFS algorithm, we can rest assured that as soon as we reach the target, the path that leads to the target is the shortest. This is also the rationale for why we should employ the BFS strategy rather than DFS.

Algorithm

In the canonical implementation of the BFS algorithm, we would employ a queue data structure to maintain the order of exploration. Each element in the queue normally contains two pieces of information: the current position and the distance traveled so far from the starting point.

However, in our problem here, we need another piece of information, which is the remaining quota that we can use to eliminate the obstacles.

Together with the coordinates, the obstacle elimination quota uniquely constitutes a state during our BFS exploration. For example, in the following graph, we demonstrate two different paths between the start cell and an intermediate cell.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_bfs_two_paths.png)

Without the obstacle elimination quota, we would only visit the intermediate cell once, while only one of the two paths can lead us to the target, since in one of the paths we don't have sufficient quota to get through. Therefore, it is critical to keep the quota information, so that we can revisit the same cell with different quotas.

Implementation

There are several ways to implement a BFS algorithm. We provide a template to do so in the Queue and Stack Explore Card.

In this section, we provide some sample implementations together with the tweak we mentioned in the above section.

We can break down the algorithm into the following steps:

1. The main body of the BFS algorithm consists of a loop around a queue, as well as a set called seen which keeps track of all the states visited along the way so that we don't visit the same state twice. A state refers to a unique combination of coordinates and the remaining quota.

2. At each iteration of the loop, we pop out one element from the queue. The element contains the distance from the starting point as well as the current state which includes the current coordinate and the remaining quotas to eliminate obstacles.

3. Within the same iteration, we evaluate the next moves starting from the popped element. Each move results in a new state and the state is valid if it is within the grid boundaries and has not been visited before. Each valid state is pushed into the queue for later iterations. Meanwhile, we also need to mark the state as visited by adding the state to the seen states set.

4. At any iteration, if we have reached our target, we can return immediately from the loop.

Note: before running the BFS traversal, we also perform a quick check to see if we have sufficient quotas to take the most direct path from start to finish regardless of the number of obstacles. If so, the shortest distance to reach the target is the Manhattan distance between the starting cell and the target cell.


In [3]:
from collections import deque
class Solution:
    def shortestPath(self, grid, k):
        rows, cols = len(grid), len(grid[0])

        target = (rows-1, cols-1)

        if k >= rows+cols-2:
            return rows+cols-2


        # (row, col, remaining quota)
        state = (0, 0, k)

        queue = deque([(0, state)])

        seen = set([state])

        while queue:
            steps, (row, col, k) = queue.popleft()

            if (row, col) == target:
                return steps
            
            for new_row, new_col in [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]:
                if (0<=new_row<rows) and (0<=new_col<cols):
                    new_elimination = k - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_elimination)

                    if new_elimination>=0 and new_state not in seen:
                        seen.add(new_state)
                        queue.append((steps+1, new_state))
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))





6


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

Time Complexity: O(N \cdot K)O(N⋅K)

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And for each cell, at most, it will be visited KK times, with different quotas of obstacle elimination.

Thus, the overall time complexity of the algorithm is O(N \cdot K)O(N⋅K).

Space Complexity: O(N \cdot K)O(N⋅K)

We used a queue to maintain the order of visited states. In the worst case, the queue will contain the majority of the possible states that we need to visit, which in total is N \cdot KN⋅K as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the visited states along the way. Same as the queue, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).

Approach 2: A* (A Star) Algorithm

Intuition

In the above BFS approach, one might notice that when at any specific position, we would systematically explore the surrounding neighbors in all four directions, due to the nature of BFS.

However, the action might seem conterintuitive or sub-optimal. Since the destination is located in the lower-right corner of the grid, in order to find the shortest path, the optimal directions to explore should be either right or down, rather than left or up.

![img](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/Figures/1293/1293_next_steps.png)

As depicted in the above image, the optimal steps to explore should be the ones in green (right and down), rather than the ones in orange.

Therefore, one idea to improve our BFS approach is to prioritize exploring the most promising directions at each step. Through prioritization, we can speed up the algorithm by reducing the time spent exploring less promising paths.

This idea leads us to the A* search algorithm, which is yet another classic path finding algorithm that uses a heuristic.

Note: we cannot exclude (or prune) those less promising directions, otherwise we might miss the correct path because sometimes we have to take a detour in order to reach the destination.

Algorithm

A* (pronounced as A star) is also known as an informed search algorithm or best-first search. Because at each step of exploration, it makes the best and informed decision on the next steps, i.e. it prioritizes the steps that are the most promising. Specifically, this prioritization strategy can be expressed as A* selects a path that minimizes the following function:f(n) = g(n) + h(n)f(n)=g(n)+h(n)

nn: a specific step during the exploration.

g(n)g(n): the cost to reach the step nn. Here, the cost refers to the distance traveled so far to the step nn.

h(n)h(n): a heuristic estimation on the cost to reach the destination from the step nn. Here, the cost refers to the distance ahead.

f(n)f(n): the estimated total cost to reach the destination if one takes the step nn.

With the defined function, A* algorithm has a deterministic way to evaluate each potential step, and then make what it believes to be the best decision at each step.

The problem boils down to defining the above functions for our scenario, in order to apply the A* algorithm. Specifically, g(n)g(n) would be the number of steps that one has taken to reach nn. And h(n)h(n) would be the Manhattan distance from nn to the destination, which is the shortest path to reach the destination.

The most important property of our heuristic h(n)h(n) function is that the function should be admissible, i.e. it never overestimates the cost. Otherwise, it could not guarantee that the path we find is the shortest one.

To understand the admissible property, let us take a metaphor. In a football tournament, we want to select the best team in the end. If we overestimate the incompetence of a team, i.e. we downplay the potential of the team, we might prematurely disqualify or ignore the team. As a result, we may predict that a not-so-good team will win the championship, while in reality, the team we disqualified or ignored happens to be the best team.

Implementation

The A* algorithm provides a more optimized path selection strategy, on top of the BFS approach. Therefore, we can implement the A* algorithm while keeping the bulk of our previous BFS approach intact. We will still use a queue to keep track of the order of visits. And we will still use a set to keep track of the visited states so that we do not revisit any previously explored paths.

Additionally, here are the modifications that we will make:

* Rather than using a normal queue, we use a priority queue to store the order of visits. The order of visits is based on the estimated total cost function f(n)f(n) that we defined. With the priority queue, the step that has potentially the lowest cost will be visited first.

* For each element in the queue, we add one more piece of information which is the estimated total cost to reach the destination at each step. This estimation will be used to prioritize each potential next step.

* We add another heuristic condition that allows us to determine the length of the shortest path without exploration. The condition is as follows:

    * At any step, if the remaining quota to eliminate the obstacles is larger than the length of the estimated shortest path (i.e. manhattan distance between the current step and the destination), then the length of the remaining path is the manhattan distance.

    * The condition can also be interpreted as if we have sufficient capacity to remove any obstacle along the way, we will simply take the shortest path to reach the destination, without the need for further exploration.

    * We apply the condition at the beginning of each iteration of the loop.


Complexity Analysis

Let NN be the number of cells in the grid, and KK be the quota to eliminate obstacles.

* Time Complexity: O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K))

We conduct a BFS traversal in the grid. In the worst case, we will visit each cell in the grid. And each cell can be visited at most KK times, with different quotas of obstacle elimination. Therefore, the total number of visits would be N \cdot KN⋅K.

For each visit, we perform one push and one pop operation in the priority queue, which takes O\big(\log{(N \cdot K)} \big)O(log(N⋅K)) time.

Thus, the overall time complexity of the algorithm is O\big(N \cdot K \cdot \log{(N \cdot K)} \big)O(N⋅K⋅log(N⋅K)).

Although the upper bound for the time complexity of the this algorithm is higher than the previous BFS approach, on average, the A* algorithm will outperform the previous BFS approach when there exists any relatively direct path from the source to the target.

* Space Complexity: O(N \cdot K)O(N⋅K)

We use a queue to maintain the order of visited states. In the worst case, the queue could contain the majority of the possible states that we must visit, which in total is N \cdot KN⋅K, as we discussed in the time complexity analysis. Thus, the space complexity of the queue is O(N \cdot K)O(N⋅K).

Other than the queue, we also used a set variable (named seen) to keep track of all the states we visited along the way. Again, the space complexity of this set is also O(N \cdot K)O(N⋅K).

To sum up, the overall space complexity of the algorithm is O(N \cdot K)O(N⋅K).






In [6]:
import heapq
class Solution:
    def shortestPath(self, grid, k):

        rows, cols = len(grid), len(grid[0])
        target = (rows - 1, cols - 1)

        def manhattan_distance(row, col):
            return target[0] - row + target[1] - col

        # (row, col, remaining_elimination)
        state = (0, 0, k)

        # (estimation, steps, state)
        # h(n) = manhattan distance,  g(n) = 0
        queue = [(manhattan_distance(0, 0), 0, state)]
        seen = set([state])

        while queue:
            estimation, steps, (row, col, remain_eliminations) = heapq.heappop(queue)

            # we can reach the target in the shortest path (manhattan distance),
            #   even if the remaining steps are all obstacles
            remain_min_distance = estimation - steps
            if remain_min_distance <= remain_eliminations:
                return estimation

            # explore the four directions in the next step
            for new_row, new_col in [(row, col + 1), (row + 1, col), (row, col - 1), (row - 1, col)]:
                # if (new_row, new_col) is within the grid boundaries
                if (0 <= new_row < rows) and (0 <= new_col < cols):
                    new_eliminations = remain_eliminations - grid[new_row][new_col]
                    new_state = (new_row, new_col, new_eliminations)

                    # if the next direction is worth exploring
                    if new_eliminations >= 0 and new_state not in seen:
                        seen.add(new_state)
                        new_estimation = manhattan_distance(new_row, new_col) + steps + 1
                        heapq.heappush(queue, (new_estimation, steps + 1, new_state))

        # did not reach the target
        return -1

# Driver Code
if __name__ == "__main__":
    grid = [[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]]
    k = 1
    print(Solution().shortestPath(grid, k))

6


# 2. Step-By-Step Directions From a Binary Tree Node to Another

You are given the root of a binary tree with n nodes. Each node is uniquely assigned a value from 1 to n. You are also given an integer startValue representing the value of the start node s, and a different integer destValue representing the value of the destination node t.

Find the shortest path starting from node s and ending at node t. Generate step-by-step directions of such path as a string consisting of only the uppercase letters 'L', 'R', and 'U'. Each letter indicates a specific direction:

* 'L' means to go from a node to its left child node.
* 'R' means to go from a node to its right child node.
* 'U' means to go from a node to its parent node.

Return the step-by-step directions of the shortest path from node s to node t.

Example 1:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg1.png)
```
Input: root = [5,1,2,3,null,6,4], startValue = 3, destValue = 6
Output: "UURL"
Explanation: The shortest path is: 3 → 1 → 5 → 2 → 6.
```

Example 2:
![img](https://assets.leetcode.com/uploads/2021/11/15/eg2.png)
```
Input: root = [2,1], startValue = 2, destValue = 1
Output: "L"
Explanation: The shortest path is: 2 → 1.
```

## Approach 1

* Build directions for both start and destination from the root.
    * Say we get "LLRRL" and "LRR".

* Remove common prefix path.
    * We remove "L", and now start direction is "LRRL", and destination - "RR"

* Replace all steps in the start direction to "U" and add destination direction.
    * The result is "UUUU" + "RR".

In [9]:
class Node:
    def __init__(self, value):
        self.val = value
        self.left, self.right = None, None

class Solution:
    def getDirections(self, root, startValue, destValue):
        def find(n, value, path):
            if n.val == value:
                return True
            if n.left and find(n.left, value, path):
                path+= "L"
            elif n.right and find(n.right, value, path):
                path+= "R"
            return path 
        s, d = [], []
        s = find(root, startValue, s)
        d = find(root, destValue, d)

        while len(s) and len(d) and s[-1]==d[-1]:
            s.pop()
            d.pop()
        return "".join("U" * len(s)) + "".join(reversed(d))

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))


UURL


## Approach 2: LCA


In [10]:
class Solution:
    def getDirections(self, root, startValue, destValue):
        
        def lca(node): 
            """Return lowest common ancestor of start and dest nodes."""
            if not node or node.val in (startValue , destValue): return node 
            left, right = lca(node.left), lca(node.right)
            return node if left and right else left or right
        
        root = lca(root) # only this sub-tree matters
        
        ps = pd = ""
        stack = [(root, "")]
        while stack: 
            node, path = stack.pop()
            if node.val == startValue: ps = path 
            if node.val == destValue: pd = path
            if node.left: stack.append((node.left, path + "L"))
            if node.right: stack.append((node.right, path + "R"))
        return "U"*len(ps) + pd

# Driver Code
if __name__ == "__main__":
    root = Node(5)
    root.left = Node(1)
    root.left.left = Node(3)
    root.right = Node(2)
    root.right.left = Node(6)
    root.right.right = Node(4)
    startValue = 3
    destValue = 6
    print(Solution().getDirections(root, startValue, destValue))

UURL


# 3. Race Car

Your car starts at position 0 and speed +1 on an infinite number line. Your car can go into negative positions. Your car drives automatically according to a sequence of instructions 'A' (accelerate) and 'R' (reverse):

* When you get an instruction 'A', your car does the following:
```python
position += speed
speed *= 2
```
* When you get an instruction 'R', your car does the following:
```python
If your speed is positive then speed = -1
otherwise speed = 1
```
Your position stays the same.

For example, after commands "AAR", your car goes to positions 0 --> 1 --> 3 --> 3, and your speed goes to 1 --> 2 --> 4 --> -1.

Given a target position target, return the length of the shortest sequence of instructions to get there.

 
```
Example 1:

Input: target = 3
Output: 2
Explanation: 
The shortest instruction sequence is "AA".
Your position goes from 0 --> 1 --> 3.
Example 2:

Input: target = 6
Output: 5
Explanation: 
The shortest instruction sequence is "AAARA".
Your position goes from 0 --> 1 --> 3 --> 7 --> 7 --> 6.
```

I -- BFS solution

Well, the BFS solution is straightforward: we can keep track of all the possible positions of the racecar after n instructions (n = 0, 1, 2, 3, 4, ...) and return the smallest n such that the target position is reached. Naive BFS will run at O(2^n) since for each position we have two choices: either accelerate or reverse. Further observations reveal that there may be overlapping among intermediate states so we need to memorize visited states (each state is characterized by two integers: car position and car speed). However, the total number of unique states still blows up for large target positions (because the position and speed can grow unbounded), so we need further pruning of the search space.

II -- DP solution

DP solution works by noting that after each reverse, the car's speed returns to 1 (the sign can be interpreted as the direction of the speed). So we can redefine the problem in terms of the position of the car while leave out the speed: let T(i) be the length of the shortest instructions to move the car from position 0 to position i, with initail speed of 1 and its direction pointing towards position i. Then our original problem will be T(target), and the base case is T(0) = 0. Next we need to figure out the recurrence relations for T(i).

Note that to apply the definition of T(i) to subproblems, the car has to start with speed of 1, which implies we can only apply T(i) right after the reverse instruction. Also we need to make sure the direction of the initial speed when applying T(i) is pointing towards the final target position.

However, we don't really know how many accelerate instructions there should be before the reverse instruction, so theoretically we need to try all possible cases: zero A, one A, two A, three A, ... and so on. For each case, we can obtain the position of the car right before the reverse instruction, which will be denoted as j = 2^m - 1, with m the number of A's. Then depending on the relation between i and j, there will be three cases:

j < i: the reverse instruction is issued before the car reaches i. In this case, we cannot apply the definition of T(i) to the subproblems directly, because even though the speed of the car returns to 1, its direction is pointing away from the target position (in this case position i). So we have to wait until the second reverse instruction is issued. Again, we don't really know how many accelerate instructions there should be in between these two reverse instructions, so we will try each of the cases: zero A, one A, two A, three A, ..., etc. Assume the number of A is q, then the car will end up at position j - p right before the second reverse instruction, where p = 2^q - 1. Then after the second reverse instruction, our car will start from position j - p with speed of 1 and its direction pointing towards our target position i. Since we want the length of the total instruction sequence to be minimized, we certainly wish to use minimum number of instructions to move the car from j - p to i, which by definition will be given by T(i-(j-p)) (note that in the definition of T(i), we move the car from position 0 to position i. If the start position is not 0, we need to shift both the start and target positions so that the start position is aligned with 0). So in summary, for this case, the total length of the instruction will be given by: m + 1 + q + 1 + T(i-(j-p)), where m is the number of A before the first R, q is the number of A before the second R, the two 1's correspond to the two R's, and lastly T(i-(j-p)) is the length of instructions moving the car from position j - p to the target position i.

j == i: the target position is reached without any reverse instructions. For this case, the total length of the instruction will be given by m.

j > i: the reverse instruction is issued after the car goes beyond i. In this case, we don't need to wait for a second reverse instruction, because after the first reverse instruction, the car's speed returns to 1 and its direction will be pointing towards our target position i. So we can apply the definition of T(i) directly to the subproblem, which will be T(j-i). Note that not only do we need to shift the start and target positions, but also need to swap them as well as the directions. So for this case, the total length of the instructions will be given by m + 1 + T(j-i).

Our final answer for T(i) will be the minimum of the above three cases.

**Time and Space Complexity:**
Both the top-down DP and bottom-up DP run at O(target * (log(target))^2) with O(target) space. However, the top-down DP may be slightly more efficient as it may skip some of the intermediate cases that must be computed explicitly for the bottom-up DP. Though the nominal time complexity are the same, both DP solutions will be much more efficient in practice compared to the BFS solution, which has to deal with (position, speed) pairs and their keys for hashing, etc.

In [14]:
# bfs 
import collections
class Solution:
    def racecar(self, target: int) -> int:
        
        #1. Initialize double ended queue as 0 moves, 0 position, +1 velocity
        queue = collections.deque([(0, 0, 1)])
        while queue:
            
            # (moves) moves, (pos) position, (vel) velocity)
            moves, pos, vel = queue.popleft()

            if pos == target:
                return moves
            
            #2. Always consider moving the car in the direction it is already going
            queue.append((moves + 1, pos + vel, 2 * vel))
            
            #3. Only consider changing the direction of the car if one of the following conditions is true
            #   i.  The car is driving away from the target.
            #   ii. The car will pass the target in the next move.  
            if (pos + vel > target and vel > 0) or (pos + vel < target and vel < 0):
                queue.append((moves + 1, pos, -vel / abs(vel)))

# Driver Code
if __name__ == "__main__":
    target = 3
    print(Solution().racecar(target))   

2


In [12]:
# dp solution
class Solution:
    def __init__(self):
        self.dp = {0: 0}
    def racecar(self, t):
        if t in self.dp:
            return self.dp[t]
        n = t.bit_length()  
        print(f"n = {n}")
        if 2**n - 1 == t:
            self.dp[t] = n
        else:
            self.dp[t] = self.racecar(2**n - 1 - t) + n + 1
            for m in range(n - 1):
                self.dp[t] = min(self.dp[t], self.racecar(t - 2**(n - 1) + 2**m) + n + m + 1)
        return self.dp[t]

# Driver Code
if __name__ == "__main__":
    t = 3
    print(Solution().racecar(t))


n = 2
2


# 4. Find Leaves of Binary Tree

Given the root of a binary tree, collect a tree's nodes as if you were doing this:

Collect all the leaf nodes.
Remove all the leaf nodes.
Repeat until the tree is empty.
 

Example 1:
![img](https://assets.leetcode.com/uploads/2021/03/16/remleaves-tree.jpg)
```
Input: root = [1,2,3,4,5]
Output: [[4,5,3],[2],[1]]
Explanation:
[[3,5,4],[2],[1]] and [[3,4,5],[2],[1]] are also considered correct answers since per each level it does not matter the order on which elements are returned.
Example 2:

Input: root = [1]
Output: [[1]]
```

Approach 1: DFS (Depth-First Search) with sorting

Intuition

The order in which the elements (nodes) will be collected in the final answer depends on the "height" of these nodes. The height of a node is the number of edges from the node to the deepest leaf. The nodes that are located in the ith height will be appear in the ith collection in the final answer. For any given node in the binary tree, the height is obtained by adding 1 to the maximum height of any children. Formally, for a given node of the binary tree \text{root}root, it's height can be represented as

$$\text{height(root)} = \text{1} + \text{max(height(root.left), height(root.right))}$$

Where $\text{root.left}$ and $\text{root.right}$ are left and right children of the root respectively

Algorithm

In our first approach, we'll simply traverse the tree recursively in a depth first search manner using the function int getHeight(node), which will return the height of the given node in the binary tree. Since height of any node depends on the height of it's children node, hence we traverse the tree in a post-order manner (i.e. height of the childrens are calculated first before calculating the height of the given node). Additionally, whenever we encounter a null node, we simply return -1 as it's height.

Next, we'll store the pair (height, val) for all the nodes which will be sorted later to obtain the final answer. The sorting will be done in increasing order considering the height first and then the val. Hence we'll obtain all the pairs in the increasing order of their height in the given binary tree.

Complexity Analysis

Time Complexity: Assuming NN is the total number of nodes in the binary tree, traversing the tree takes O(N)O(N) time. Sorting all the pairs based on their height takes O(N \log N)O(NlogN) time. Hence overall time complexity of this approach is O(N \log N)O(NlogN)

Space Complexity: O(N)O(N), the space used by pairs. solution also requires O(N)O(N) space however the output does not count towards the space complexity.


In [25]:
class Solution:
    def __init__(self):
        self.pairs = []

    def getHeight(self, root):
        if not root:
            return -1
        leftHeight = self.getHeight(root.left)
        rightHeight = self.getHeight(root.right)

        currHeight = max(leftHeight, rightHeight) + 1

        self.pairs.append((currHeight, root.val))
        return currHeight
    
    def findLeaves(self, root):
        self.getHeight(root)
        self.pairs.sort(key=lambda x: x[0])
        # print(self.pairs)
        n = len(self.pairs)
        height = 0
        i=0
        solution = []
        while i<n:
            nums = []
            while i<n and self.pairs[i][0] == height:
                nums.append(self.pairs[i][1])
                i += 1
            solution.append(nums)
            # print(solution)
            height += 1
        return solution

# Driver Code
if __name__ == "__main__":
    root = Node(1)
    root.left = Node(2)
    root.right = Node(3)
    root.left.left = Node(4)
    root.left.right = Node(5)
    print(Solution().findLeaves(root))


[[4, 5, 3], [2], [1]]


# 5.  Remove All Ones With Row and Column Flips

You are given an m x n binary matrix grid.

In one operation, you can choose any row or column and flip each value in that row or column (i.e., changing all 0's to 1's, and all 1's to 0's).

Return true if it is possible to remove all 1's from grid using any number of operations or false otherwise.

Example 1:
![img](https://assets.leetcode.com/uploads/2022/01/03/image-20220103191300-1.png)
```
Input: grid = [[0,1,0],[1,0,1],[0,1,0]]
Output: true
Explanation: One possible way to remove all 1's from grid is to:
- Flip the middle row
- Flip the middle column
```

Explanation
```
I honestly don't know how to categorize this problem. It seems like a Math problem to me. Once you understand the logic, the implementation is simple.
Basically the "pattern" of each row should be the same, by pattern, I mean following:
001100 and 001100 are the same pattern
001100 and 110011 (the invert of original) are the same pattern
Only in above situation, one matrix can be converted to all zero
Intuition?
Believe it or not, I draw a couple examples to test it out and suddenly it becomes obvious
I guess it's good habit to get your hands dirty :)
```



In [31]:
class Solution:
    def removeOnes(self, grid):
        r1, r1_invert = grid[0], [1-val for val in grid[0]]
        for i in range(1, len(grid)):
            if grid[i] != r1 and grid[i] != r1_invert:
                return False
        return True

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True


In [30]:
class Solution:
    def removeOnes(self, grid):
        return all(grid[i] == grid[0] or grid[i] == [1-val for val in grid[0]] for i in range(len(grid)))

# Driver Code
if __name__ == "__main__":
    grid = [[0,1,0],[1,0,1],[0,1,0]]
    print(Solution().removeOnes(grid))

True
